# Datetime queries

To index a file containing a list of JSON objects into Elasticsearch, the most efficient method is to use the Elasticsearch Bulk API via Python. This approach allows you to index multiple documents in a single request, significantly improving performance compared to indexing documents individually.

First, ensure you have the Elasticsearch Python client installed. You can install it using pip:

In Elasticsearch, finding documents that match an exact date requires using a range query that specifies both a start and end time for that single date, or a match query if the field mapping and query string are configured correctly to handle only the date portion. 


If I query for records in Elasticsearch using the date filter such as gt '2024-12-31', do records with datetimes occurring during 2024-12-31 but before 2025-01-01 included?

In [73]:
# !conda install -y elasticsearch

In [1]:
!conda list elasticsearch

# packages in environment at /opt/anaconda3:
#
# Name                     Version          Build            Channel
elasticsearch              9.2.0            py313hca03da5_0


Next, create a Python script that reads the JSON file, processes the list of objects, and uses the `helpers.bulk()` method to index them. Here’s a complete example:

In [2]:
import json
import pandas as pd
from elasticsearch import Elasticsearch, helpers
from datetime import datetime

In [3]:
# Connect to Elasticsearch
# copy http_ca.crt from ES container (config/cert/http_ca.crt)
client = Elasticsearch("http://localhost:9200",
    # ca_certs="/Users/blauerbock/.ssh/http_ca.crt",
    # basic_auth=("elastic", "giraffe") 
)

# Define the index name
index_name = "datetime"
print(client)

<Elasticsearch(['http://localhost:9200'])>


Greater than a date

The elastic_transport.ObjectApiResponse class is a response type used in the Elasticsearch Python client for API responses that return a JSON object.
 It inherits from Generic[_ObjectBodyType] and ApiResponse[Dict[str, Any]], providing a structured way to access the response body and metadata.
 The class includes a body property that provides a user-friendly view into the raw response with type hints if applicable, allowing access to the JSON data as a dictionary-like object.
 Additionally, it includes a meta property that contains metadata such as the HTTP status code, headers, duration, and the node that handled the request.
 This class is used across various Elasticsearch API methods in versions 8.0.0a1 and later, including operations like update_by_query_rethrottle, delete, put_autoscaling_policy, and others.
 The response structure ensures compatibility with both old-style dictionary access (e.g., resp['hits']['hits']) and modern attribute-style access via the body property, although the latter is recommended for future compatibility.

In [24]:
conditions_column = []
compare_dates_column = []
result_dates_column = []
conditions = ["gt"]
compare_dates = ["2023-12-31", "2023-12-31T00:00:00Z"]
for condition in conditions:
    for compare_date in compare_dates:
    
        query_body = {
            "query": {
                "range": {
                    "submission_date": {
                        condition: compare_date
                    }
                } 
            }
        }

        result = client.search(index="datetime", body=query_body)
        # print(type(result.body))  # dict
        result_body = result.body
        # print(json.dumps(result.body, indent=2))
        # print(result.body['hits']['hits'])
        hits = result.body['hits']['hits']
        for hit in hits:
            # print(f"{hit=}")
            print(f"{hit['_source']['submission_date']=}")
            
        # json_result = json.dumps(result, indent=2)
        # result_dates = [hits['_source']['submission_date'] for hit in hits]
        result_dates = sorted(result_dates)
        for result_date in result_dates:
            conditions_column.append(condition)
            compare_dates_column.append(compare_date)
            result_dates_column.append(result_date)

df_gt = pd.DataFrame(zip(conditions_column, compare_dates_column, result_dates_column), columns = ['conditions', 'criterion', 'selected'])


hit['_source']['submission_date']='2024-01-01'
hit['_source']['submission_date']='2024-01-01T00:00:00Z'
hit['_source']['submission_date']='2024-01-01T12:00:00.00Z'
hit['_source']['submission_date']='2024-01-01T23:59:59Z'
hit['_source']['submission_date']='2024-01-01T23:59:59.99Z'
hit['_source']['submission_date']='2024-01-01T23:59:59.9999Z'
hit['_source']['submission_date']='2024-01-02'


NameError: name 'result_dates' is not defined

In [82]:
df_gt

,conditions,criterion,selected
0,gt,2023-12-31,2023-12-31T12:00:00Z
1,gt,2023-12-31,2023-12-31T23:59:59.9999Z
2,gt,2023-12-31,2023-12-31T23:59:59Z
3,gt,2023-12-31,2024-01-01
4,gt,2023-12-31,2024-01-01T00:00:00Z
5,gt,2023-12-31,2024-01-01T12:00:00.00Z
6,gt,2023-12-31,2024-01-01T23:59:59.9999Z
7,gt,2023-12-31,2024-01-01T23:59:59.99Z
8,gt,2023-12-31,2024-01-01T23:59:59Z
9,gt,2023-12-31,2024-01-02


## Equal to a date
TlsError: TLS error caused by: TlsError(TLS error caused by: SSLError([SSL] record layer failure (_ssl.c:1028)))
Solution: use http, not https, when configuring client.

In [83]:
conditions_column = []
compare_dates_column = []
result_dates_column = []
conditions = ["equal to (gte and lte)"]
compare_dates = ["2023-12-31", "2023-12-31T00:00:00Z"]
for condition in conditions:
    for compare_date in compare_dates:
    
        query_body = {                
          "query": {
            "range": {
              "submission_date": {
                "gte": compare_date,
                "lte": compare_date
              }
            }
          }
        }
    
        result = client.search(index="datetime", body=query_body)   
        result_dates = [hit['_source']['submission_date'] for hit in hits]
        result_dates = sorted(result_dates)
        for result_date in result_dates:
            conditions_column.append(condition)
            compare_dates_column.append(compare_date)
            result_dates_column.append(result_date)

df_eq = pd.DataFrame(zip(conditions_column, compare_dates_column, result_dates_column), columns = ['conditions', 'criterion', 'selected'])


Date math

https://www.elastic.co/docs/reference/elasticsearch/rest-apis/common-options#date-math

In [87]:
conditions_column = []
compare_dates_column = []
result_dates_column = []
conditions = ["equal to (gte and lte)"]
compare_dates = ["2023-12-31"]  #, "2023-12-31T00:00:00Z"]
for condition in conditions:
    for compare_date in compare_dates:        
        query_body = {
          "query": {
            "range": {
              "submission_date": {
                "gte": "2023-12-31||/d",
                "lte": "2023-12-31||/d"
                # "time_zone": "America/New_York"
              }
            }
          }
        }
        print(query_body)
        result = client.search(index="datetime", body=query_body)   
        result_dates = [hit['_source']['submission_date'] for hit in hits]
        result_dates = sorted(result_dates)
        for result_date in result_dates:
            conditions_column.append(condition)
            compare_dates_column.append(compare_date)
            result_dates_column.append(result_date)

df_eq2 = pd.DataFrame(zip(conditions_column, compare_dates_column, result_dates_column), columns = ['conditions', 'criterion', 'selected'])


{'query': {'range': {'submission_date': {'gte': '2023-12-31||/d', 'lte': '2023-12-31||/d'}}}}


In [88]:
df_eq2

,conditions,criterion,selected
0,equal to (gte and lte),2023-12-31,2023-12-31T12:00:00Z
1,equal to (gte and lte),2023-12-31,2023-12-31T23:59:59.9999Z
2,equal to (gte and lte),2023-12-31,2023-12-31T23:59:59Z
3,equal to (gte and lte),2023-12-31,2024-01-01
4,equal to (gte and lte),2023-12-31,2024-01-01T00:00:00Z
5,equal to (gte and lte),2023-12-31,2024-01-01T12:00:00.00Z
6,equal to (gte and lte),2023-12-31,2024-01-01T23:59:59.9999Z
7,equal to (gte and lte),2023-12-31,2024-01-01T23:59:59.99Z
8,equal to (gte and lte),2023-12-31,2024-01-01T23:59:59Z
9,equal to (gte and lte),2023-12-31,2024-01-02


In [ ]:
# You can also use date math for this, by rounding down the gte value to the start of the day and implicitly rounding the lte value to the end of the day. 
json



GET your_index/_search
{
  "query": {
    "range": {
      "your_date_field": {
        "gte": "2023-01-01||/d",
        "lte": "2023-01-01||/d",
        "time_zone": "America/New_York"
      }
    }
  }
}

In [52]:
df

,conditions,criterion,selected
0,gt,2023-12-31,2023-12-31T12:00:00Z
1,gt,2023-12-31,2023-12-31T23:59:59.9999Z
2,gt,2023-12-31,2023-12-31T23:59:59Z
3,gt,2023-12-31,2024-01-01
4,gt,2023-12-31,2024-01-01T00:00:00Z
5,gt,2023-12-31,2024-01-01T12:00:00.00Z
6,gt,2023-12-31,2024-01-01T23:59:59.9999Z
7,gt,2023-12-31,2024-01-01T23:59:59.99Z
8,gt,2023-12-31,2024-01-01T23:59:59Z
9,gt,2023-12-31,2024-01-02


In [39]:
conditions = []
compare_dates = []
hits = result['hits']['hits']
print(hits[0]['_source']['submission_date'])
sd = [hit['_source']['submission_date'] for hit in hits]
ssd = sorted(sd)
print(ssd)
for result_date in ssd:
    conditions.append(condition)
    compare_dates.append(compare_date)

df = pd.DataFrame(zip(conditions, compare_dates, ssd), columns = ['conditions', 'criterion', 'selected'])


#print((*hits).submission_date)
# print(result['hits']['hits'][:]['submission_date')

2023-12-31T12:00:00Z
['2023-12-31T12:00:00Z', '2023-12-31T23:59:59.9999Z', '2023-12-31T23:59:59Z', '2024-01-01', '2024-01-01T00:00:00Z', '2024-01-01T12:00:00.00Z', '2024-01-01T23:59:59.9999Z', '2024-01-01T23:59:59.99Z', '2024-01-01T23:59:59Z', '2024-01-02']


In [40]:
df

,conditions,criterion,selected
0,gt,2023-12-31T00:00:00Z,2023-12-31T12:00:00Z
1,gt,2023-12-31T00:00:00Z,2023-12-31T23:59:59.9999Z
2,gt,2023-12-31T00:00:00Z,2023-12-31T23:59:59Z
3,gt,2023-12-31T00:00:00Z,2024-01-01
4,gt,2023-12-31T00:00:00Z,2024-01-01T00:00:00Z
5,gt,2023-12-31T00:00:00Z,2024-01-01T12:00:00.00Z
6,gt,2023-12-31T00:00:00Z,2024-01-01T23:59:59.9999Z
7,gt,2023-12-31T00:00:00Z,2024-01-01T23:59:59.99Z
8,gt,2023-12-31T00:00:00Z,2024-01-01T23:59:59Z
9,gt,2023-12-31T00:00:00Z,2024-01-02


In [14]:
# Function to load JSON data from a file
def load_json_data(filename):
    with open(filename, "r", encoding="utf-8") as file:
        return json.load(file)

# Load the JSON data
# data_path = r"/Users/blauerbock/workspaces/python-workout/recursive_json_parsing/users.json"
data_path = r"/Users/blauerbock/workspaces/python-workout/recursive_json_parsing/data/users.json"
json_data = load_json_data(data_path)  # "data.json")  # Replace with your file path

# Prepare the list of documents for bulk indexing
doc_list = []
for i, doc in enumerate(json_data):
    # Optionally add a timestamp
    doc["timestamp"] = datetime.now().isoformat()
    # Optionally set a custom _id
    doc["_id"] = i
    doc_list.append(doc)

In [5]:
print(doc_list[:2])

[{'id': 1, 'firstName': 'Emily', 'lastName': 'Johnson', 'maidenName': 'Smith', 'age': 29, 'gender': 'female', 'email': 'emily.johnson@x.dummyjson.com', 'phone': '+81 965-431-3024', 'username': 'emilys', 'password': 'emilyspass', 'birthDate': '1996-5-30', 'image': 'https://dummyjson.com/icon/emilys/128', 'bloodGroup': 'O-', 'height': 193.24, 'weight': 63.16, 'eyeColor': 'Green', 'hair': {'color': 'Brown', 'type': 'Curly'}, 'ip': '42.48.100.32', 'address': {'address': '626 Main Street', 'city': 'Phoenix', 'state': 'Mississippi', 'stateCode': 'MS', 'postalCode': '29112', 'coordinates': {'lat': -77.16213, 'lng': -92.084824}, 'country': 'United States'}, 'macAddress': '47:fa:41:18:ec:eb', 'university': 'University of Wisconsin--Madison', 'bank': {'cardExpire': '05/28', 'cardNumber': '3693233511855044', 'cardType': 'Diners Club International', 'currency': 'GBP', 'iban': 'GB74MH2UZLR9TRPHYNU8F8'}, 'company': {'department': 'Engineering', 'name': 'Dooley, Kozey and Cronin', 'title': 'Sales Man

In [15]:
# Perform bulk indexing
try:
    print("Attempting to index the list of docs using helpers.bulk()")
    response = helpers.bulk(
        client,
        doc_list,
        index=index_name,
        # doc_type="_doc"  # Use "_doc" for Elasticsearch 6.0+
    )
    print("Bulk indexing completed:", response)
except Exception as e:
    print("Error during bulk indexing:", e)

Attempting to index the list of docs using helpers.bulk()
Bulk indexing completed: (30, [])


python elasticsearch Attempting to index the list of docs using helpers.bulk()
Error during bulk indexing: Connection error caused by: ConnectionError(Connection error caused by: ProtocolError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))))

This script assumes your JSON file contains a list of objects, such as:
```json
[
  {"name": "Alice", "age": 30},
  {"name": "Bob", "age": 25}
]
```
The `helpers.bulk()` method expects a list of dictionaries, where each dictionary represents an Elasticsearch document. You can include additional fields like `_id` or `timestamp` as needed. The `doc_type` parameter is set to `_doc` to comply with Elasticsearch 6.0+ standards, as the `doc_type` field has been deprecated 

Alternatively, you can use the `curl` command with the `_bulk` endpoint if you prefer a command-line approach. The JSON file must be formatted with alternating lines: one line for the action/metadata (e.g., `{"index": {"_id": "1"}}`) and one line for the document body. For example:

```json
{"index": {"_id": "1"}}
{"name": "Alice", "age": 30}
{"index": {"_id": "2"}}
{"name": "Bob", "age": 25}
```

Then use the following command:

```bash
curl -XPOST "http://localhost:9200/your_index_name/_bulk" --data-binary @data.json
```
This method requires the file to be in the NDJSON (Newline Delimited JSON) format, where each line is a valid JSON object 

Both approaches are effective, but using the Python `helpers.bulk()` method is generally preferred for its ease of integration, error handling, and flexibility in data preprocessing 